# Trabájo Practico 2 - Métodos Computacionales

### Camila Cauzzo, Catalina Dolhare

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import time
import random

In [2]:
def abrirImagenesEscaladas( carpeta, escala=32 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [3]:
def balancear_datos(imagenes1, imagenes2):
    cantidad_imagenes = min(len(imagenes1), len(imagenes2))
    imagenes1 = imagenes1[:cantidad_imagenes]
    imagenes2 = imagenes2[:cantidad_imagenes]

    imagenes = imagenes1 + imagenes2

    return imagenes

### Ejercicio 1

Calcular las derivadas parciales de la funci ́on L con respecto a w y b 

In [4]:
def L_w(i, w, b, d):
    t0 = np.tanh(np.dot(w, i) + b)
    return (1 - t0**2) * (((1 + t0) / 2) - d) * i

In [5]:
def L_b(i, w, b, d):
    t0 = np.tanh(np.dot(w, i) + b)
    return (1 - t0**2) * (((1 + t0) / 2) - d)

In [6]:
def f(i, w, b):
    tan = np.tanh(np.dot(w, i) + b)
    return (tan + 1) / 2

In [7]:
def dL_dw(w, b, imagenes, diagnosticos):
    gradiente = np.zeros_like(w)
    for i, imagen in enumerate(imagenes):
        gradiente += L_w(imagen, w, b, diagnosticos[i])
    return gradiente / len(imagenes)

In [8]:
def dL_db(w, b, imagenes, diagnosticos):
    gradiente = 0
    for i, imagen in enumerate(imagenes):
        gradiente += L_b(imagen, w, b, diagnosticos[i])
    return gradiente / len(imagenes)

In [9]:
def func_L(w, b, imagenes, diagnosticos):
    error = 0
    for i, imagen in enumerate(imagenes):
        prediccion = f(imagen, w, b)
        error += (prediccion - diagnosticos[i])**2
    return error / len(imagenes)

### Ejercicio 2

Implementar el método de descenso por gradiente y optimizar los parámetros de la función f para
el conjunto de datos de entrenamiento

In [16]:
def gradiente_descendiente(imagenes1, imagenes2):
    imagenes_normal = abrirImagenesEscaladas(imagenes1)
    diagnosticos_normal = [0] * len(imagenes_normal)

    imagenes_neumonia = abrirImagenesEscaladas(imagenes2)
    diagnosticos_neumonia = [1] * len(imagenes_neumonia)

    imagenes = balancear_datos(imagenes_normal, imagenes_neumonia)
    diagnosticos = diagnosticos_normal + diagnosticos_neumonia

    w_t = np.zeros(imagenes[0].shape[0])
    b_t = 0
    alpha = 0.001

    MAX_ITER = 1000
    TOLERANCIA = 1e-8
    iteracion = 0

    while iteracion <= MAX_ITER:
        print("Iteración: ", iteracion, "- Mínimo alcanzado hasta el momento: ", func_L(w_t, b_t, imagenes, diagnosticos))

        gradient_dw = dL_dw(w_t, b_t, imagenes, diagnosticos)
        gradient_db = dL_db(w_t, b_t, imagenes, diagnosticos)

        w_t_mas_1 = w_t - alpha * gradient_dw
        b_t_mas_1 = b_t - alpha * gradient_db

        dif = func_L(w_t_mas_1, b_t_mas_1, imagenes, diagnosticos) - func_L(w_t, b_t, imagenes, diagnosticos)

        if abs(dif) < TOLERANCIA:
            break

        w_t = w_t_mas_1
        b_t = b_t_mas_1

        iteracion += 1
    
    return w_t, b_t, imagenes, diagnosticos

In [17]:
w_opt, b_opt, imagenes, diagnosticos = gradiente_descendiente("chest_xray/train/NORMAL", "chest_xray/train/PNEUMONIA")
print("El mínimo local ocurre en w = ", w_opt, "b = ", b_opt, " y L vale: ", func_L(w_opt, b_opt, imagenes, diagnosticos))

Iteración:  0 - Mínimo alcanzado hasta el momento:  0.25
Iteración:  1 - Mínimo alcanzado hasta el momento:  0.2497635922203209
Iteración:  2 - Mínimo alcanzado hasta el momento:  0.2495312324593706
Iteración:  3 - Mínimo alcanzado hasta el momento:  0.2493019422638432
Iteración:  4 - Mínimo alcanzado hasta el momento:  0.2490750031769165
Iteración:  5 - Mínimo alcanzado hasta el momento:  0.24884988791325668
Iteración:  6 - Mínimo alcanzado hasta el momento:  0.2486262096598789
Iteración:  7 - Mínimo alcanzado hasta el momento:  0.24840368476342134
Iteración:  8 - Mínimo alcanzado hasta el momento:  0.24818210529011162
Iteración:  9 - Mínimo alcanzado hasta el momento:  0.24796131885909337
Iteración:  10 - Mínimo alcanzado hasta el momento:  0.24774121382945333
Iteración:  11 - Mínimo alcanzado hasta el momento:  0.24752170842502558
Iteración:  12 - Mínimo alcanzado hasta el momento:  0.2473027427537651
Iteración:  13 - Mínimo alcanzado hasta el momento:  0.24708427295365398
Iteración

### Ejercicio 3

Calcular el error cuadrático durante la optimización para el conjunto de entrenamiento y para
el conjunto de testing. Generar las visualizaciones correspondientes.

In [ ]:
def error_cuadratico_medio(imagenes, diagnostico, w, b):
    error_total = 0
    for i in len(imagenes):
        error = (f(imagenes[i], w, b) - diagnostico[i])**2
        error_total += error

    return error_total/len(imagenes)

Generamos un grafico para ver como varia el error a medida que avanzan las iteraciones

In [ ]:
plt.plot(errores, range(len(errores)))
plt.xlabel("Iteraciones")
plt.ylabel("Error")
plt.title("Error cuadrático medio en el entrenamiento")
plt.show()

NameError: name 'errores' is not defined

##### Como es con testing? Hay que hacer gradiente descendiente en estos datos tambien y ver el error o hay que usar el w y b optimo del train y ver que tanto error da cuando se aplica f a cada imagen?

### Ejercicio 4

##### Otra vez misma duda. Hay que hacer gradiente descendiente para test o optimizar b y w y ver como queda f para cada imagen de test? Como veo la convergencia???????????

Analizar el impacto del parámetro α en la convergencia del método. Tomar un rango
de 5 valores posibles y analizar la convergencia para el conjunto de testing para los distintos valores de α.

El parametro alpha es muy importante a la hora de encontrar el minimo. Este determina cuanto vamos a movernos en dirección al gradiente. Si este es muy chico, encontrar el minimo va a tardar muchos iteración y, dependiendo de el valor maximo, es posible que no se encuentre. Si se elige un valor muy grande, es posible que nos pasemos del valor minimo local.

In [ ]:
alpha = [0.0000001, 0.0001, 0.01, 1, 10]
for a in alpha:
    w_opt, b_opt = gradiente_descendiente(imagenes, a)
    print("Alpha: ", a, "- L = ", L(imagenes, w_opt, b_opt), " - Error cuadrático medio: ", error_cuadratico_medio(imagenes, w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  60.67761335170323
w: [-3.06642722e-07 -4.30690102e-07 -4.30929655e-07 ...  1.15433243e-07
  1.55135632e-07  1.69193019e-07]
b: 0.4999963656900931
L nuevo: 53.07365823098891
L viejo: 60.67761335170323
diferencia L: 7.603955120714318
Iteración:  1 - Mínimo alcanzado hasta el momento:  53.07365823098891
w: [-1.81921134e-07 -4.30774881e-07 -4.34814820e-07 ...  4.02058690e-07
  5.08398319e-07  5.49468263e-07]
b: 0.4999940570316326
L nuevo: 50.42672157639585
L viejo: 53.07365823098891
diferencia L: 2.6469366545930626
Iteración:  2 - Mínimo alcanzado hasta el momento:  50.42672157639585
w: [ 3.04826898e-07 -5.81612696e-08 -6.84936028e-08 ...  8.09714086e-07
  1.00032642e-06  1.07714779e-06]
b: 0.4999930844972514
L nuevo: 49.771893872707835
L viejo: 50.42672157639585
diferencia L: 0.6548277036880137
Iteración:  3 - Mínimo alcanzado hasta el momento:  49.771893872707835
w: [9.30270381e-07 4.59395080e-07 4.41864898e-07 ... 1.25962754e-06
 1.540

KeyboardInterrupt: 

### Ejercicio 5

¿Cómo impacta el tamaño del escalado de las imágenes en la efectividad del método? ¿Y en el
tiempo de cómputo?. Realizar los experimentos y gráficos acordes para estudiar estas limitaciones

In [ ]:
escalado = [16, 32, 64, 128, 256, 512]

for e in escalado:
    imagenes_normal = abrirImagenesEscaladas("chest_xray/small_train/NORMAL", e)
    imagenes_pneumonia = abrirImagenesEscaladas("chest_xray/small_train/PNEUMONIA", e)
    imagenes = [imagenes_normal, imagenes_pneumonia]

    tiempo_inicial = time.time()
    w_opt, b_opt = gradiente_descendiente(imagenes, 0.001)
    tiempo_final = time.time()
    print("Tiempo de ejecución: ", tiempo_final-tiempo_inicial)
    print("Escala: ", e, "- L = ", L(imagenes, w_opt, b_opt), "- Error cuadrático medio: ", error_cuadratico_medio(imagenes, w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  50.0
Iteración:  1 - Mínimo alcanzado hasta el momento:  47.37579165693172
Iteración:  2 - Mínimo alcanzado hasta el momento:  54.40794117059423
Iteración:  3 - Mínimo alcanzado hasta el momento:  96.60520972069014
Iteración:  4 - Mínimo alcanzado hasta el momento:  92.70278868357126
Iteración:  5 - Mínimo alcanzado hasta el momento:  69.75197786150365
Iteración:  6 - Mínimo alcanzado hasta el momento:  94.28938051439916
Iteración:  7 - Mínimo alcanzado hasta el momento:  79.86667296177323
Iteración:  8 - Mínimo alcanzado hasta el momento:  74.54716144111538
Iteración:  9 - Mínimo alcanzado hasta el momento:  87.4604925217846
Iteración:  10 - Mínimo alcanzado hasta el momento:  37.70919849237352
Iteración:  11 - Mínimo alcanzado hasta el momento:  37.5991346070351
Iteración:  12 - Mínimo alcanzado hasta el momento:  72.82041175355346
Iteración:  13 - Mínimo alcanzado hasta el momento:  87.60263641799033
Iteración:  14 - Mínimo alcanza

### Ejercicio 6

Para el valor de α que tenga mejor valor de convergencia, generar la matriz de confusión y analizar
brevemente la efectividad del método.